---
title: "Computation for Linguists"
subtitle: "SpaCy & Semantic Mappings"
date: "November 5, 2025"
author: "Dr. Andrew M. Byrd"
format:
  revealjs:
    css: header_shrink.css
    theme: beige
    slide-number: true
    center: true
    toc: true
    toc-title: "Plan for the Day"
    toc-depth: 1
jupyter: python3
editor: source
---


# Review

-   What did you learn last time?

## Recap from Last Time

``` python
import spacy
nlp = spacy.load("en_core_web_sm")

text = "Dr. Byrd's students can't wait to analyze PIE roots!"

doc = nlp(text)
[t.text for t in doc]
```

## Recap from Last Time

``` python
import spacy
import pandas as pd

# Load English model
nlp = spacy.load("en_core_web_sm")

text = "Dr. Byrd's students can't wait to analyze PIE roots!"
doc = nlp(text)

# Create list of dicts, one per token
data = []
for t in doc:
    data.append({
        "text": t.text,
        "lemma": t.lemma_,
        "POS": t.pos_,
        "tag": t.tag_,
        "stop": t.is_stop,
        "is_punct": t.is_punct
    })

# Make DataFrame
df = pd.DataFrame(data)
print(df)
```

## Recap from Last Time

``` python
import spacy

content_lemmas = [t.lemma_.lower() for t in doc
                  if not (t.is_stop or t.is_punct or t.is_space or t.like_num)]
```

## Recap from Last Time

``` python
from collections import Counter
import pandas as pd

freq = Counter(content_lemmas)
df_freq = (pd.DataFrame(freq.items(), columns=["lemma", "count"])
           .sort_values("count", ascending=False))
df_freq.head(10)
```

## Recap from Last Time

``` python
import matplotlib.pyplot as plt

top10 = df_freq.head(10)
plt.figure()
plt.bar(top10["lemma"], top10["count"])
plt.title("Top 10 Content Lemmas")
plt.xlabel("Lemma")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
```

## Review Activity

-   Copy the below string, and after filtering out stopwords and non-words, identify the top 10 most frequent words.

``` python
getty = "Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battle-field of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.But, in a larger sense, we can not dedicate—we can not consecrate—we can not hallow—this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us—that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion—that we here highly resolve that these dead shall not have died in vain—that this nation, under God, shall have a new birth of freedom—and that government of the people, by the people, for the people, shall not perish from the earth."
```

# Using spaCy for Syntactic Processing

## Sentence Segmentation

``` python
text2 = ("President Pitzer, Mr. Vice President, Governor Connally, ladies and "
         "gentlemen: I am delighted to be here today. "
         "We meet in an hour of change and challenge.")
doc2 = nlp(text2)
[s.text for s in doc2.sents]
```

## Dependency Parse (head, relation)

-   [Full Glossary of spaCy abbreviations](https://github.com/explosion/spaCy/blob/master/spacy/glossary.py)

``` python
doc = nlp("The quick brown fox jumps over the lazy dog.")
[(t.text, t.dep_, t.head.text) for t in doc]
```

## Visualize Dependencies (displaCy)

``` python
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("The quick brown fox jumps over the lazy dog.")

svg = displacy.render(doc, style="dep")  # returns SVG/HTML markup
with open("syntax_tree.svg", "w", encoding="utf-8") as f:
    f.write(svg)
```

## Visualize Dependencies (displaCy)

![](syntax_tree.svg)

## Activity: Dependencies

- Visualize the following sentence using `displaCy`

``` python
doc  = "This spaCy library is too dang powerful."
```

## Visualize Dependencies (displaCy)

![](activity_2_syntax_tree.svg)


## Extracting Verbs + Direct Objects

- We can also identify specific syntactic relations within our sentences, such as all nouns that are direct objects.
  - by using the `.children` attribute, we identify all immediate syntactic dependents of a token

```python
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp("We considered the options and chose the best proposal.")
pairs = []
for tok in doc:
    if tok.pos_ == "VERB":
        dobj = [c for c in tok.children if c.dep_ == "dobj"]
        if dobj:
            pairs.append((tok.lemma_, dobj[0].text))
pairs
```

## Rule-Based Matching (verbs of violence, e.g.)

- And we can also filter out for specific groups of words that we define beforehand


In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")

violent_verbs = ["punch", "kick", "attack", "strike", "hit", "assault"]
patterns = [nlp(v) for v in violent_verbs]
matcher = PhraseMatcher(nlp.vocab, attr="LEMMA")
doc = nlp("They punched, kicked, and attacked the intruder before fleeing.")
matcher.add("VIOLENCE", patterns)
[(doc[s:e].text, doc[s:e].lemma_) for _, s, e in matcher(doc)]

# Semantic Analysis Using WordNet

## Semantic Analysis

- In the previous code block we predefined `violent_verbs`
- But what if we were able to access a library that already contained all of these?

## WordNet

- We can connect to **WordNet**, which is a *lexical database*
  - Developed at Princeton University  
  - Organizes English words into **synsets** (sets of cognitive synonyms)  

## WordNet

- Captures relationships among words:
  - **Synonyms:** *good ↔ nice*  
  - **Antonyms:** *hot ↔ cold*  
  - **Hypernyms:** *dog → animal*  
  - **Hyponyms:** *dog → poodle*  
  - **Meronyms:** *car → wheel*
  - **Entailments:** *snore → sleep* 

## Setting up WordNet

``` bash
# Installing spacy-wordnet
pip install spacy spacy-wordnet nltk

# Installing NLTK wordnet data
python -m nltk.downloader wordnet
python -m nltk.downloader omw

# Downloading English spaCy model
python -m spacy download en_core_web_sm
```

<!-- # Download an English wordnet into wn’s local DB (one time) -->
<!-- import wn -->
<!-- wn.download('oewn:2021')   # Open English WordNet 2021 -->
<!-- # (alternatives: 'ewn:2020', 'omw-en:1.4') -->
```


In [ ]:
import spacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacy_wordnet", after="tagger", config={"lang": "en"})

doc = nlp("The dog chased the cat.")
for tok in doc:
    if tok.pos_ in ("NOUN","VERB","ADJ","ADV"):
        syns = tok._.wordnet.synsets()
        lemmas = tok._.wordnet.lemmas()
        hypers = tok._.wordnet.hypernyms()
        print(tok.text, [s.definition() for s in syns[:1]])

In [ ]:
economy_domains = ['finance', 'banking']
enriched_sentence = []
sentence = nlp('I want to withdraw 5,000 euros')

# For each token in the sentence
for token in sentence:
    # We get those synsets within the desired domains
    synsets = token._.wordnet.wordnet_synsets_for_domain(economy_domains)
    if not synsets:
        enriched_sentence.append(token.text)
    else:
        lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names()]
        # If we found a synset in the economy domains
        # we get the variants and add them to the enriched sentence
        enriched_sentence.append('({})'.format('|'.join(set(lemmas_for_synset))))

# Let's see our enriched sentence
print(' '.join(enriched_sentence))
# >> I (need|want|require) to (draw|withdraw|draw_off|take_out) 5,000 euros

In [ ]:
import wn

# list all synsets for 'dog' (English noun)
wn.synsets('dog', pos='n', lang='en')

In [ ]:
import wn
for s in wn.synsets('dog', pos='n', lang='en'):
    print(f"{s}: {s.lemmas()}")     # list of lemma strings

In [ ]:
import wn

# pick an adjective sense like 'good' to find antonyms more easily
s = wn.synsets('good', pos='a', lang='en')[0]
synonyms = s.lemmas()  # lemma strings
# antonyms live on *senses* (word-in-synset); collect across them:
ants = []
for sense in s.senses():                # Sense objects
    for ant in sense.antonyms():        # antonym Senses
        ants.extend(ant.synset().lemmas())
synonyms, sorted(set(ants))

In [ ]:
import wn
s = wn.synsets('dog', pos='n', lang='en')[0]
print(s.hypernyms())     # more general
print(s.hyponyms()[:10]) # more specific (slice for brevity)
print(s.part_meronyms()) # parts (e.g., tail, paw)
print(s.part_holonyms()) # wholes (e.g., pack, kennel)

In [ ]:
import wn
wn.synsets("run", lang='en')

s = wn.synsets("run", pos='v', lang='en')[0]
print("Lemma names:", s.lemmas())
print("Definitions:", s.definitions())   # list
print("Examples:",   s.examples())       # list

In [ ]:
import wn
s = wn.synsets("dog", pos='n', lang='en')[0]
print("Hypernyms:", s.hypernyms())
print("Hyponyms:", s.hyponyms())
print("Part Meronyms:", s.part_meronyms())

In [ ]:
import wn
from wn.similarity import path, wup, lch
import wn.taxonomy as tax

dog = wn.synsets('dog', pos='n', lang='en')[0]
cat = wn.synsets('cat', pos='n', lang='en')[0]

print("path(dog,cat) =", path(dog, cat))  # 0..1 (higher=more similar)
print("wup(dog,cat)  =", wup(dog, cat))   # Wu–Palmer

# LCH needs taxonomy depth:
n_depth = tax.taxonomy_depth(wn.Wordnet('oewn:2021'), 'n')
print("lch(dog,cat)  =", lch(dog, cat, n_depth))

In [ ]:
# All hyponyms (recursive)
def all_hyponyms(synset):
    hypos = set()
    for h in synset.hyponyms():
        hypos.add(h)
        hypos |= all_hyponyms(h)
    return hypos

dog = wn.synsets('dog', pos='n', lang='en')[0]
all_hypos = all_hyponyms(dog)
len(all_hypos), list(sorted(all_hypos))[:10]

In [ ]:
# Car example
car_synsets = wn.synsets('car', pos='n', lang='en')
car = car_synsets[0]
print(car.definitions()[0])
print(car.examples())
print(car.hypernyms())
print(car.hyponyms()[:10])

In [ ]:
import spacy, wn
nlp = spacy.load("en_core_web_sm")

text = "The dogs are running near the river banks."
doc = nlp(text)

# Filter to content lemmas
content = [t for t in doc if not (t.is_stop or t.is_punct or t.is_space or t.like_num)]
lemmas  = [t.lemma_.lower() for t in content]

lemmas

In [ ]:
# Map spaCy UPOS to wn POS for better lookups
def spacy_pos_to_wn_pos(pos):
    return {'NOUN':'n','VERB':'v','ADJ':'a','ADV':'r'}.get(pos, None)

lemma_pos = [(t.lemma_.lower(), spacy_pos_to_wn_pos(t.pos_)) for t in content]
lemma_pos

In [ ]:
# Get synsets per lemma (restrict by POS when available)
synsets = []
for lemma, pos in lemma_pos:
    synsets.append(wn.synsets(lemma, pos=pos, lang='en') if pos else wn.synsets(lemma, lang='en'))
synsets

In [ ]:
# Similarity among hand-picked senses (Wu–Palmer)
from wn.similarity import wup

targets = [wn.synsets('dog', pos='n', lang='en')[0],
           wn.synsets('cat', pos='n', lang='en')[0],
           wn.synsets('bank', pos='n', lang='en')[8]]   # e.g., 'financial institution' vs. 'river bank'
for i in range(len(targets)):
    for j in range(i+1, len(targets)):
        a, b = targets[i], targets[j]
        print(a.id, b.id, wup(a, b))

## Why WordNet (for linguists)?

-   **Synsets** (sense inventories) feel familiar from semantics.
-   Clear relations: **hypernym/hyponym**, **meronym/holonym**, **antonym**.
-   Great for building *semantic fields* (e.g., motion verbs).


## First Look at Synsets

``` python
from nltk.corpus import wordnet as wn
wn.synsets("dog")  # list of senses across POS
```

## Definitions & Examples

``` python
for s in wn.synsets("dog"):
    print(s.name(), "→", s.definition(), "|", s.examples())
```

## Hypernyms & Hyponyms

``` python
dog = wn.synset("dog.n.01")
dog.hypernyms(), dog.hyponyms()[:5]
```

## Synonyms & Antonyms (lemmas)

``` python
s = wn.synset("good.a.01")
synonyms = [l.name() for l in s.lemmas()]
antonyms = [l.antonyms()[0].name() for l in s.lemmas() if l.antonyms()]
synonyms, antonyms
```

## Simple Semantic Similarity

> **Note:** This is **taxonomy-based**, not vectors. Values are rough but intuitive.

``` python
dog = wn.synset("dog.n.01")
cat = wn.synset("cat.n.01")
animal = wn.synset("animal.n.01")
print("dog ~ cat (path):", dog.path_similarity(cat))
print("dog ~ animal (path):", dog.path_similarity(animal))
print("Wu-Palmer dog~cat:", dog.wup_similarity(cat))
```

## Build a Mini Semantic Field (verbs of speech)

``` python
seed = "say"
field = set()
for s in wn.synsets(seed, pos=wn.VERB):
    field.update({h.name().split(".")[0] for h in s.hyponyms()})
sorted(list(field))[:25]
```


# All senses of "run"
synsets = wn.synsets("run")
print(f"Number of senses for 'run': {len(synsets)}")
print(synsets[:5])
```

------------------------------------------------------------------------

# Inspecting a Synset

In [ ]:
s = wn.synsets("run")[0]
print("Name:", s.name())
print("Definition:", s.definition())
print("Examples:", s.examples())

------------------------------------------------------------------------

# Semantic Relations in WordNet


In [ ]:
s = wn.synsets("dog")[0]

print("Definition:", s.definition())
print("Hypernyms (broader):", s.hypernyms())
print("Hyponyms (narrower):", s.hyponyms())
print("Meronyms (parts):", s.part_meronyms())

------------------------------------------------------------------------

# Discussion 💬

-   How does WordNet organize meaning?
-   What’s the difference between a **synset** and a **lemma**?
-   Why do NLP systems need both lexical data (like WordNet) *and* text processing tools (like spaCy)?


# Discussion 💬

-   What linguistic knowledge does spaCy rely on for lemmatization?
-   When might WordNet be more useful than spaCy, and vice versa?
-   How could you combine both in a project (e.g., linking lemmas to their WordNet synsets)?

------------------------------------------------------------------------

# Next Time ➡️

🌍 **Multilingual NLP** — applying tokenization and lemmatization to other languages (Spanish, French, German, etc.)

> Each group will analyze one language and compare results.